In [3]:
import pandas as pd

# Load datasets
global_monthly = pd.read_csv("../data/raw/Globalmonthlyandseasonal.csv")
southern_hemisphere = pd.read_csv("../data/raw/SouthernHemisphere.csv")

# Cleaning - handle missing values
global_monthly.replace("***", pd.NA, inplace=True)
southern_hemisphere.replace("***", pd.NA, inplace=True)

# Save cleaned data
global_monthly.to_csv("../data/cleaned/cleaned_globalmonthlyandseasonal.csv", index=False)
southern_hemisphere.to_csv("../data/cleaned/cleaned_southernhemisphere.csv", index=False)
